In [ ]:
using DataFrames, FreqTables, Plots, Distributions, StatsPlots, Colors
using QuadGK
gr()

# an auxiliary function to display certain tabular data
function print_summary(mat, title)
    A=DataFrame()
    A[:,:hit] = [mat[1,1] / sum(mat[1,:]) ]
    A[:,:fa] = [mat[2,1] / sum(mat[2,:])]
    println(title)
    display(A)   # not sure how to turn off the `1rows x 2 colums` title.
end

# IL027: Interdisciplinary Computer Modelling

## Cognitive Models: Signal Detection Theory

#### Henrik Singmann, Department of Psychology

Email: Henrik.Singmann@warwick.ac.uk

## Overview

This notebook provides a short introduction to calculating signal detection theory (SDT) models for binary decision data. Chapters 1 and 2 of the textbook by [Macmillan and Creelman (2005)](http://ebookcentral.proquest.com/lib/warw/detail.action?docID=234275) cover pretty much the same content as discussed here. A general introduction is given by [Abdi (2007)](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.80.5869&rep=rep1&type=pdf). A comprehensive overview is provided by [Kellen and Klauer (2018)](http://davidkellen.org/wp-content/uploads/2016/02/chapter_stevens_final.pdf). 

**Please note:** What this document and the lexture calls *discriminability* is often called *sensitivity* in the existing literature (e.g., the references given above). The problem with the term *sensitivity* is that it has a different meaning in a related context in statistics so can be ambiguous. To avoid this ambiguity, we prefer the term *discriminability* to refer to the ability of the decision maker to distinguish two stimulus classes.

### References

Abdi, H. (2007). [Signal detection theory (SDT)](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.80.5869&rep=rep1&type=pdf). In N. Salkind, Encyclopedia of measurement and statistics (pp. 886–889). Sage. 

Kellen, D., & Klauer, K. C. (2018). [Elementary Signal Detection and Threshold Theory.](http://davidkellen.org/wp-content/uploads/2016/02/chapter_stevens_final.pdf) In J. T. Wixted (Ed.), Stevens’ Handbook of Experimental Psychology and Cognitive Neuroscience (pp. 1–39). John Wiley & Sons, Inc. https://doi.org/10.1002/9781119170174.epcn505

Macmillan, N. A., & Creelman, C. D. (2005). [Detection theory: A user’s guide](http://ebookcentral.proquest.com/lib/warw/detail.action?docID=234275). Lawrence Erlbaum associates.



## Signal Detection Tasks

Signal detection theory based models are applied to data from signal detection tasks. In signal detection tasks, the task of the decision maker is to decide whether or not an event or stimulus contains a signal. That is, detection tasks are defined by two characteristics:

- Each stimulus can be unambiguously categorized as either a signal-present or a signal-absent event. We usually denote this distinction as one between two **stimulus classes**: signal or noise stimuli.
- Decision makers provide a single **response** for each stimulus, a binary decision: "signal present" or "signal absent"

The combination of stimulus class and response produces the following 2-by-2 table where columns represent the response and rows represent stimulus class. The four cells within the table provide the names of the four observable response frequencies. Frequencies plotted in bold are correct responses.

|   | "signal present"  |  "signal absent" | 
|---|---|---|
|  signal | **Hit** ($H$) |  Miss ($M$) |  
|  noise |  False Alarm ($FA$) | **Correct Rejection** ($CR$)  |  

Relative frequencies (i.e., response proportions) are obtained by dividing by the row marginals (i.e., relative frequencies are relative for each stimulus class). Thus, signal detection tasks only provide two independent data points:

$H = P$("signal present" | signal)  
$FA = P$("signal present" | noise)

The other two relative frequencies can be obtained from those two:

$M = 1 - H$  
$CR = 1 - FA$






### Actual image manipulation data


We begin with a real-life signal detection data set from a image manipulation detection experiment. The stimuli are images that are either digitally manipulated (e.g., via facetune or photoshop) or not. For each image, participants have to decide whether or not the image was manipulated or not. We input the data in such a way that it is in the 2-by-2 table:

In [ ]:
## Create dummy DataFrame to establish names of rows/columns
d = DataFrame( picture = ["manipulated", "original"], 
               response = ["manipulated", "original"] );
##Create Named Array of actual image manipulation data
d_images=freqtable(d, :picture, :response)
d_images[:, :] = [522 528; 275 775]
display(d_images)

From this table it is easy to calculate Hits and False Alarms as described above.

In [ ]:
## Hit:
round(d_images[1,1] ./ sum(d_images[1,:]), digits = 2)

In [ ]:
## FA:
round(d_images[2,1] / sum(d_images[2,:]), digits=2)

### Made up Data

To better understand what type of information is provided by the 2-by-2 table of signal detection tasks, we imagine we have data from three different decision makers (participants), `p_a`, `p_b`, and `p_c`. At first, we are interested which of these three participants shows the highest performance in terms of distinguishing the two stimulus classes. For this, we calculate Hits and False Alarms from the tables.

In [ ]:
p_a=freqtable(d, :picture, :response)
p_a[:,:] = [80 20; 20 80]
print_summary(p_a, "p_a");

In [ ]:
p_b=freqtable(d, :picture, :response)
p_b[:,:] = [60 40; 40 60]
print_summary(p_b, "p_b");

In [ ]:
p_c=freqtable(d, :picture, :response)
p_c[:,:] = [60 40; 0 100]
print_summary(p_c, "p_c");

`p_a` has more hits and less false alarms than `p_b`. Thus, `p_a` appears to be better than `p_b` in distinguishing signal from noise stimuli. 

However, the relationship between `p_a` and `p_c` is not as simple. Whereas `p_a` has more hits than `p_c`, `p_a` also has more false alarms than `p_c`. This makes clear that the ability to distinguish between the two stimulus classes affects both, hits and false alarms.

To better understand the interplay of the two frequencies, we calculate two other measures from the frequency table:

- The probability of being correct: $C =$ (Hits + Correct Rejections) / Total N
- The probability of making a "signal" response: $S =$ (Hits + False Alarms) / Total N

These two measures show that there are at least two latent cognitive processes at work in detection tasks, *dicriminability* (i.e., how well a decision maker distinguishes signal from noise stimuli) and *respons bias* (i.e., propensity of decision maker to prefer one response option over the other).

## Signal Detection Theory (SDT)

Signal detection theory (SDT) provides principled approach to disentangle the two latent process in signal detection tasks,  dicriminability and respons bias. 

SDT assumes that the decision maker bases their decision on a continuous latent strength dimension: the subjective strength of the signal (e.g., in an image manipulation detection task, the feeling of how manipulated an image is). This latent strength is noisy and represented by random variables or probability distributions, one distribution for signal stimuli and one for noise stimuli. Here, we follow the common convenience assumption and assume the distribution are Gaussian (i.e., follow the normal distribution). SDT thus assumes that each stimulus that is encountered provides the decision maker with one value on the strength dimension.

To transform the continuous subjective strength signal, the decision maker establishes a response criterion, $k$, on the continuous strength dimension. If the value of an item exceeds the criterion the decision maker responds with "signal", otherwise with "no signal".

The folllowing is a plot of the Gaussian SDT model:


In [ ]:
### Basic plot of SDT

xx=[-10:0.01:10]
y1=Normal(-0.5)
y2=Normal(0.5)
plot(xx,y1,fill=(1,0,:gray),fillalpha=(0.5),linecolor = nothing,xlims=(-4,4),ylims=(0,0.8),
     xlabel="Latent Strength",ylabel="Density",title="Gaussian SDT",legend=nothing,xticks=false)
plot!(xx,y2,fill=(1,0,:gray),fillalpha=(0.5),linecolor = nothing)
annotate!([(-1.5,0.4, Plots.text("Noise", 14, :center)), (1.5, 0.4, Plots.text("Signal", :center, 14))])
vline!([0.3],linewidth=3,linecolor=:black)
annotate!([(-1.2,0.7, Plots.text("\"No Signal\"", 16, :center)), (1.5, 0.7, Plots.text("\"Signal\"", :center, 16))])
annotate!([(1.75,0.55, Plots.text("Response\ncriterion k", 12, :center))])
plot!([1.1,0.3],[0.55,0.55], arrow = true, linewidth=2, linecolor=:black)

Because the integral of the full probability distributions correspond to 1 and each probability distribution has an area on each side of the criterion, the area under the curve (i.e., the integral) on each side of the response criterion corresponds to the predicted response probability for each stimulus class.

To show this more clearly we can split the plot into two, one plot for each stimulus type.

In [ ]:
### split of basic SDT plot
p1 = plot(xx,y2,fill=(1,0,:gray),fillalpha=(0.5),linecolor = nothing,xlims=(-4,4),ylims=(0,0.8),
     xlabel="Latent Strength",ylabel="Density",title="Gaussian SDT (signal distribution only)",
    legend=nothing,xticks=false)
annotate!([(1.5, 0.4, Plots.text("Signal", :center, 14))])
vline!([0.3],linewidth=3,linecolor=:black)
annotate!([(-1.2,0.7, Plots.text("\"No Signal\"", 16, :center)), (1.5, 0.7, Plots.text("\"Signal\"", :center, 16))])
annotate!([(1.75,0.55, Plots.text("Response\ncriterion k", 12, :center))])
plot!([1.1,0.3],[0.55,0.55], arrow = true, linewidth=2, linecolor=:black)

p2 = plot(xx,y1,fill=(1,0,:gray),fillalpha=(0.5),linecolor = nothing,xlims=(-4,4),ylims=(0,0.8),
     xlabel="Latent Strength",ylabel="Density",title="Gaussian SDT (noise distribution only)",
    legend=nothing,xticks=false)
annotate!([(-1.5, 0.4, Plots.text("Noise", :center, 14))])
vline!([0.3],linewidth=3,linecolor=:black)
annotate!([(-1.2,0.7, Plots.text("\"No Signal\"", 16, :center)), (1.5, 0.7, Plots.text("\"Signal\"", :center, 16))])
annotate!([(1.75,0.55, Plots.text("Response\ncriterion k", 12, :center))])
plot!([1.1,0.3],[0.55,0.55], arrow = true, linewidth=2, linecolor=:black)

plot(p1, p2, layout = (2, 1), size = (600, 700))

The upper plot shows the signal distribution only. The area on the left side of the response criterion corresponds to the probability of misses, the area on the right side of the criterion to the probability of hits.

Likewise, the lower panel shows the noise distribution only. Here, the area on the left side of the response criterion corresponds to the probability of correct rejections and the area on the right side of the criterion to the probability of false alarms.

### Disentangling discriminability and response bias

SDT allows disentangling discriminability and bias by changing the relative position of the two distributions as well as the position of the response criterion. The position of the response criterion determines the response bias.

The following plot exemplifies this using two response criterion, one liberal (left) and one conservative (right). The position of the distribution is as in the previous plots (i.e., discriminability as before), but by moving the response criterion from the left (i.e., liberal) to the right (i.e., conservative) position, both hits and false alarms decrease.

In [ ]:
plot(xx,y1,fill=(1,0,:gray),fillalpha=(0.5),linecolor = nothing,xlims=(-4,4),ylims=(0,0.8),
     xlabel="Latent Strength",ylabel="Density",title="Gaussian SDT (low discriminability)",legend=nothing,xticks=false)
plot!(xx,y2,fill=(1,0,:gray),fillalpha=(0.5),linecolor = nothing)
annotate!([(-1.5,0.4, Plots.text("Noise", 14, :center)), (1.5, 0.4, Plots.text("Signal", :center, 14))])
vline!([0.5],linewidth=3,linecolor=:black,linestyle=:dash)
vline!([-0.4],linewidth=3,linecolor=:black,linestyle=:dot)
annotate!([(1.4,0.7, Plots.text("Conservative\nResponse\ncriterion k", 12, :center))])
annotate!([(-1.2,0.7, Plots.text("Liberal\nResponse\ncriterion k", 12, :center))])

The positions of the two response distributions determine the discriminability of the decision maker. In the following we see the same two response criteria as in the plots just above, but the response distributions are further apart. The consequence of the larger distance between the two response distributions is that we have both more hits, but less false alarms. The two different response criteria however have the same qualitative effect as above. The more liberal, the more false alarms and more hits.

In [ ]:
### Disentangling discriminability and bias

plot(xx,Normal(-1.5),fill=(1,0,:gray),fillalpha=(0.5),linecolor = nothing,xlims=(-4,4),ylims=(0,0.8),
     xlabel="Latent Strength",ylabel="Density",title="Gaussian SDT (high discriminability)",
    legend=nothing,xticks=false)
plot!(xx,Normal(1.25),fill=(1,0,:gray),fillalpha=(0.5),linecolor = nothing)
annotate!([(-2.25,0.4, Plots.text("Noise", 14, :center)), (1.95, 0.4, Plots.text("Signal", :center, 14))])
vline!([0.5],linewidth=3,linecolor=:black,linestyle=:dash)
vline!([-0.4],linewidth=3,linecolor=:black,linestyle=:dot)
annotate!([(1.4,0.7, Plots.text("Conservative\nResponse\ncriterion k", 12, :center))])
annotate!([(-1.2,0.7, Plots.text("Liberal\nResponse\ncriterion k", 12, :center))])

### SDT Formally

As described above, the predicted response probabilities (i.e., hits and false alarms as well as their complements, misses and correct rejections, respectively) correspond to the area under the curve on either side of the response criterion for each evidenc distribution. For example, the probabilities of hits, $H$, and false alarms, $FA$, is given by

$$\begin{align}
H = P(E_s > k) &= \int_{k}^{\infty} f_s(x) \, dx,\\
FA = P(E_n > k) &= \int_{k}^{\infty} f_n(x) \, dx,
\end{align}$$

where $E_s$ and $E_n$ represent the latent strength or evidence random variables for signal and noise stimuli, respectively. Furthermore, $f_s(x)$ and $f_n(x)$ represent the probability densitiy functions (PDF) for these random variables.

We make the usual assumptions that the distributions are Gaussian (i.e., follow a normal distribution) and that mean and variance of the noise distribution are fixed at 0 and 1, respectively. 

Now, imagine for a moment we knew that the criterion $k$ for a decision maker were 0.64. Then, we can simply take the integral of the normal PDF from thius value (i.e., 0.64) to infinity to calculate the predicted probability of false alarms. Here, we do so numerically using `quadgk()` (i.e., we approximate the integral numerically using a method called adaptive Gauss-Kronrod quadrature):

In [ ]:
quadgk(x -> pdf(Normal(0, 1), x), 0.64, Inf, rtol=1e-3)

This results in a predicted false alarm rate of 0.26, or roughly the rate of false alarms observed for our example image manipulation data. Sidenote: We could have also written `quadgk(x -> pdf(Normal(), x), 0.64, Inf, rtol=1e-3)` and would have gotten the same result, as 0 and 1 are the default values for the normal distribution.

Furthermore, imagine we knew that the mean of the signal distribution were also 0.64. This would result in a predicted hit rate of 0.5, or again roughly the hit rate obtained from our actual image manipulation data.

In [ ]:
quadgk(x -> pdf(Normal(0.64, 1), x), 0.64, Inf, rtol=1e-3)

What this shows is that once we know the mean of the signal distribution as well as the criterion location, calculating predicted response proportions is simple. The question is of course how we can go the other way (i.e., from data to model parameters). 

To understand how this is done, we first need to understand one mathematical relationship; the relationship between the [probability density function (PDF)](https://en.wikipedia.org/wiki/Probability_density_function) and the [cumulative distribution function (CDF)](https://en.wikipedia.org/wiki/Cumulative_distribution_function) for a random variable. Whereas the PDF evaluated at $t$ gives the relative likelihood of the random variable at $t$, the CDF evaluated at $t$ gives the probability that the random variable takes on a value of $t$ or smaller. Mathematically, the CDF, $F(t)$, gives the integral from negative infinity to $t$ of the PDF, $f(t)$:

$$F(t) = \int_{-\infty}^{t} f(x) \, dx$$

Thus, for the case of false alarms, the predictions can also simply be obtained via the (probability) complement of the CDF. This is because per definition a PDF integrates to 1, and we want the integral from $k$ to $\infty$ insteads of from $-\infty$ to $k$,

$$FA = \int_{k}^{\infty} f_n(x) \, dx = 1 - \int_{-\infty}^{k} f_n(x) \, dx = 1 - F(k).$$


In [ ]:
1 - cdf(Normal(0, 1), 0.64)

From this relationship, we can also derive how to obtain $k$ from $FA$. The only bit we need is the inverse of the CDF, $F(k)$, the function $F(k)^{-1}$. $F(k)^{-1}$ is also known as the [quantile function](https://en.wikipedia.org/wiki/Quantile_function).

$$\begin{align}
FA &= 1 - F(k)\\
FA - 1 &= - F(k)\\
1 - FA &= F(k)\\
F(1 - FA)^{-1} &= k
\end{align}$$

In code the last line looks as follows:

In [ ]:
quantile(Normal(0, 1), 1 - 0.26)

In case the distribution is the normal (i.e., Gaussian) distribution, which is symmetric around 0 for $\mu = 0$, this can also be epxressed as 
$$-\Phi(FA)^{-1}= k,$$
where $\Phi(\cdot)^{-1}$ refers to the quantile function of the normal distribution (the literatur also sometimes refers to it as $z(\cdot)$).

In [ ]:
-quantile(Normal(0, 1), 0.26)

Following a similar logic, the mean of the signal distribution, $\mu_s$ or alternativelys $d'$ (d-prime), can also be derived. For the normal distribution that results in
$$\mu_s = d' = \Phi(H)^{-1} - \Phi(FA)^{-1}.$$

Note that the steps for this equation also require certain properties for the underlying probability distribution (i.e., invariance to mean shifts and symmetry around the mean).

The following figure shows PDF and CDF of the standard normal distribution (i.e., $\mu = 0$, $\sigma^2 = 1$). As discussed above, the right plot gives the integral (i.e., the area under the curve) of the left plot from minus infinity to the value on the x-axis.

In [ ]:
### Cumulative Distribution function

p_pdf = plot(xx, pdf.(Normal(), xx),xlims=(-3.5,3.5),ylims=(-0.02,0.42),xlabel="Latent Strength", 
    ylabel="Cumulative Probability",title="PDF: Gaussian Distribution",
    xticks=true,legend=false,linecolor=:black, lw=2)
hline!([0,1],linecolor=:gray, lw=2)
p_cdf = plot(xx, cdf.(Normal(), xx),xlims=(-3.5,3.5),ylims=(-0.02,1.02),xlabel="Latent Strength", 
    ylabel="Cumulative Probability",title="CDF: Gaussian Distribution",
    xticks=true,legend=false,linecolor=:black, lw=2)
hline!([0,1],linecolor=:gray, lw=2)
plot(p_pdf, p_cdf, layout = (1, 2), size = (800, 350))

## EVSDT in Practice

We will now use both formulas to calculate SDT measures.

$$d' = \Phi(H)^{-1} - \Phi(FA)^{-1}$$
$$k = -\Phi(FA)^{-1}$$

In [ ]:
## P(Hit):
H = d_images[1,1] / sum(d_images[1,:])

In [ ]:
## P(FA):
FA = d_images[2,1] / sum(d_images[2,:])

In [ ]:
## d'
quantile.(Normal(),H)-quantile.(Normal(),FA)

In [ ]:
## k:
-quantile.(Normal(),FA)

Whereas $d'$ is easy to interpret as a measure of discriminability, $k$ is less so as it depends on $d'$. Alternative measures for the response bias are $c$ and $c'$ for which 0 indicates no bias, values below 0 indicate a liberal response bias, and values above 0 indicate a conservative response bias.

$$c = -0.5 * (z(H)+z(F))$$
$$c' = c/d' = -0.5 * [(z(H)+z(F)) / (z(H)-z(F))]$$

In [ ]:
## c
-0.5*(quantile.(Normal(),H)+quantile.(Normal(),FA))

In [ ]:
## c'
-0.5* ((quantile.(Normal(),H) + quantile.(Normal(),FA)) / (quantile.(Normal(),H) - quantile.(Normal(),FA)))

We can also plot the model implied by our data.

In [ ]:
## Plot our SDT model
plot(xx,Normal(0),fill=(1,0,:gray),fillalpha=(0.5),linecolor = nothing,xlims=(-4,4),ylims=(0,0.8),
     xlabel="Latent Strength",ylabel="Density",title="EVSDT for Manipulated Image Data",
    legend=nothing)
plot!(xx,Normal(quantile.(Normal(),H)-quantile.(Normal(),FA)),fill=(1,0,:gray),fillalpha=(0.5),linecolor = nothing)
annotate!([(-0.85,0.4, Plots.text("Noise", 14, :center)), (1.55, 0.4, Plots.text("Signal", :center, 14))])
vline!([-quantile.(Normal(),FA)],linewidth=3,linecolor=:black)
annotate!([(-0.7,0.7, Plots.text("\"No Signal\"", 16, :center)), (1.7, 0.7, Plots.text("\"Signal\"", :center, 16))])
annotate!([(1.35,0.55, Plots.text("Response\ncriterion k", 12, :center))])

### Receiver Operating Characteristic (ROC) Function

The starting point for SDT is the observation that hits and false alarms jointly determine discriminability and response bias. Thus, any change in either of the two process should affect both observable response proportions. The ROC function is derived from the SDT model and allows describing the relationship between the response proportion assuming fixed discriminability $d'$, but varying $k$.

We begin with the formula for $d'$ from above:
$$\begin{align*}
d' &= \Phi(H)^{-1} - \Phi(FA)^{-1} \\
\end{align*}$$

We then want to solve for $H$:
$$\begin{align*}
d' &= \Phi(H)^{-1} - \Phi(FA)^{-1} & | \;& +\Phi(FA)^{-1} \\
d' + \Phi(FA)^{-1} &= \Phi(H)^{-1} & | \;& \Phi(\cdot) \\
\Phi(d' + \Phi(FA)^{-1}) &= H & 
\end{align*}$$

We can use this to plot the predicted hit rate as a function of FA for fixed $d'$:


In [ ]:
plot(0:0.01:1, cdf.(Normal(), 1 .+ quantile.(Normal(),0:0.01:1)), 
     lw=2, linecolor=:black, xlims=(0,1), ylims=(0,1), 
     xlabel="False Alarm Rate",ylabel="Hit Rate",title="ROC for d' = 1",
     legend=nothing,size=(450,450))
plot!([0,1],[0,1],linestyle=:dash,linecolor=:gray, lw=2)

To calculate individual values we dould do as follows:

In [ ]:
dp = 1  ## d'
FA = 0.3 
cdf(Normal(), dp + quantile(Normal(), FA))

ROC functions can be useful in case we have multiple sets of hits and false alarms. In such as situation the substantive question of interest is often whether or not the different sets show the same discriminability or not. With the ROC function at hand, this question can be reformulated as to whether or not the different sets fall on the same ROC curve or not.

## Potential Study Topics

- Ekman, O'Sullivan, and Frank (1999) videotaped men either lying or telling the truth about social issues on which they held strong beliefs, and played the tapes to seven groups of observers, different types of law enforcement agents and psychologists. The proportions of correct responses to lies and truths are given in their table 2. Are their differences in discriminability and response bias among the seven groups?
- The ROC function plots the hitrate ($H$) as a function of the false alarm rate ($FA$) given fixed discriminability (i.e., fixing $d'$ to a constant). Given the terminology used in this document the ROC function could also be described as the *isodiscriminability* function (and is known as the *isosensitivity* function in the literature). This indicates that one could also consider the existence of an isobias function, a function of hits as a function of false alarms given fixed response bias. Given the three available measures for bias, $k$, $c$, and $c'$, how does the isobias function look like and which appears to be the most reasonable in a given situation?


### References

- Ekman, P., O’Sullivan, M., & Frank, M. G. (1999). A Few Can Catch a Liar. Psychological Science, 10(3), 263–266. https://doi.org/10.1111/1467-9280.00147




### Advanced Topics

- Kellen, D., Klauer, K. C., & Singmann, H. (2012). On the measurement of criterion noise in signal detection theory: The case of recognition memory. Psychological Review, 119(3), 457–479. https://doi.org/10.1037/a0027727

- Kellen, D., Winiger, S., Dunn, J. C., & Singmann, H. (preprint). Testing the Foundations of Signal Detection Theory in Recognition Memory. https://doi.org/10.31234/osf.io/p5rj9